# Detecting Sentiment Based on OpenAi Solution

https://openai.com/blog/unsupervised-sentiment-neuron/

This solution consists on using a multiplicative LSTM (mLSTM) to simply predict the next charactere on product reviews. After training the recurrent network, use it's hidden units to train a classifier.

In my case, I lack the resources and time available in OpenAi (as long as the knowledge), so I am going to do my best with a few days of training on a GeForce GTX 1060 instead of "one month across four NVIDIA Pascal GPUs [...] processing 12,500 characters per second."

Instead of using amazon reviews, I will use a portuguese language review dataset obtained from https://www.kaggle.com/olistbr/brazilian-ecommerce.

This project was made for the natural language processing discipline, UFPELs computer science course.

In [1]:
import os
import sys
import pandas as pd
raw_data = "../data/raw/"

reviews = pd.read_csv(os.path.join(raw_data, "olist_order_reviews_dataset.csv"), 
                      usecols=["review_id", "review_score", "review_comment_title", "review_comment_message"]).fillna("")
reviews = reviews[reviews.review_comment_message != ""]
reviews.head()

,review_id,review_score,review_comment_title,review_comment_message
3,e64fb393e7b32834bb789ff8bb30750e,5,,Recebi bem antes do prazo estipulado.
4,f7c4243c7fe1938f181bec41a392bdeb,5,,Parabéns lojas lannister adorei comprar pela I...
9,8670d52e15e00043ae7de4c01cc2fe06,4,recomendo,aparelho eficiente. no site a marca do aparelh...
12,4b49719c8a200003f700d3d986ea1a19,4,,"Mas um pouco ,travando...pelo valor ta Boa.\r\n"
15,3948b09f7c818e2d86c9a546758b2335,5,Super recomendo,"Vendedor confiável, produto ok e entrega antes..."


In [2]:
reviews.groupby("review_score").review_id.count()

review_score
1     9179
2     2229
3     3665
4     6034
5    20646
Name: review_id, dtype: int64

# Define Multiplicative LSTM

Model of recurrent neural network used by OpenAi people (https://arxiv.org/abs/1609.07959). Model obtained from https://discuss.pytorch.org/t/implementation-of-multiplicative-lstm/2328/5.

In [3]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.nn.functional as F

class mLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, embed_size, output_size):
        super(mLSTM, self).__init__()

        self.hidden_size = hidden_size
        # input embedding
        self.encoder = nn.Embedding(input_size, embed_size)
        # lstm weights
        self.weight_fm = nn.Linear(hidden_size, hidden_size)
        self.weight_im = nn.Linear(hidden_size, hidden_size)
        self.weight_cm = nn.Linear(hidden_size, hidden_size)
        self.weight_om = nn.Linear(hidden_size, hidden_size)
        self.weight_fx = nn.Linear(embed_size, hidden_size)
        self.weight_ix = nn.Linear(embed_size, hidden_size)
        self.weight_cx = nn.Linear(embed_size, hidden_size)
        self.weight_ox = nn.Linear(embed_size, hidden_size)
        # multiplicative weights
        self.weight_mh = nn.Linear(hidden_size, hidden_size)
        self.weight_mx = nn.Linear(embed_size, hidden_size)
        # decoder
        self.decoder = nn.Linear(hidden_size, output_size)

    def forward(self, inp, h_0, c_0):
        # encode the input characters
        inp = self.encoder(inp)
        # calculate the multiplicative matrix
        m_t = self.weight_mx(inp) * self.weight_mh(h_0)
        # forget, input and output gates
        f_g = torch.sigmoid(self.weight_fx(inp) + self.weight_fm(m_t))
        i_g = torch.sigmoid(self.weight_ix(inp) + self.weight_im(m_t))
        o_g = torch.sigmoid(self.weight_ox(inp) + self.weight_om(m_t))
        # intermediate cell state
        c_tilda = torch.tanh(self.weight_cx(inp) + self.weight_cm(m_t))
        # current cell state
        cx = f_g * c_0 + i_g * c_tilda
        # hidden state
        hx = o_g * torch.tanh(cx)

        out = self.decoder(hx.view(1,-1))

        return out, hx, cx

    def init_hidden(self):
        h_0 = Variable(torch.zeros(1, self.hidden_size)).cuda()
        c_0 = Variable(torch.zeros(1, self.hidden_size)).cuda()
        return h_0, c_0

In [4]:
import random

def generate_chunk(predict_len=200, temperature=0.5):
    prime_str = "O produto"  
    hidden, cell = rnn.init_hidden()
    prime_input = char_tensor(prime_str).cuda()
    predicted = prime_str

    # prime_str é o texto inicial que o gerado irá completar
    for p in range(len(prime_str) - 1):
        _, hidden, cell = rnn(prime_input[p], hidden, cell)
    inp = prime_input[-1]
    
    for p in range(predict_len):
        output, hidden, cell = rnn(inp, hidden, cell)
        
        # Usa a temperatura para amostrar a distribuição e escolher a saída probabilísticamente
        output_dist = output.data.view(-1).div(temperature).exp()
        top_i = torch.multinomial(output_dist, 1)[0]
        
        # Adiciona o caracter predito à string de saída
        predicted_char = chr(top_i)
        predicted += predicted_char
        inp = char_tensor(predicted_char).cuda()

    return predicted

In [5]:
# Converte string para uma lista de inteiros
from unidecode import unidecode

def tensor2char(tensor, temperature=0.2):
    output_dist = output.data.view(-1).div(temperature).exp()
    top_i = torch.multinomial(output_dist, 1)[0]
    return chr(top_i)

def char_tensor(string):
    string = unidecode(string)
    tensor = torch.zeros(len(string)).long()
    for c in range(len(string)):
        try:
            tensor[c] = ord(string[c])
        except:
            print(c)
            raise
    return Variable(tensor)

print(char_tensor('The omega (Ω) symbol\n'))

tensor([ 84, 104, 101,  32, 111, 109, 101, 103,  97,  32,  40,  79,  41,  32,
        115, 121, 109,  98, 111, 108,  10])


In [6]:
# Treina sobre um exemplo (i.e. uma amostragem do texto)

def train(inp, target):
    hidden = rnn.init_hidden()
    rnn.zero_grad()
    loss = 0

    inp = inp.cuda()
    hidden, cell = rnn.init_hidden()
    for c in range(len(inp)):
        output, hidden, cell = rnn(inp[c], hidden, cell)
        loss += loss_metric(output, target[c].unsqueeze(0))

    loss.backward()
    optimizer.step()

    return loss.data.item() / len(inp)


In [7]:
import time

n_epochs = 100000
print_every = 100
embed_size = 128 # ascii representation
hidden_size = 2048
lr = 0.0001

cuda = torch.device('cuda')

rnn = mLSTM(embed_size, hidden_size, embed_size, embed_size).cuda()

optimizer = torch.optim.Adam(rnn.parameters(), lr=lr)
loss_metric = nn.CrossEntropyLoss()

start = time.time()
all_losses = []
loss_avg = 0

dataset = list(map(char_tensor, reviews.review_comment_message.values))



In [ ]:
for epoch in range(1, n_epochs + 1):
    for i, review in enumerate(dataset): 
        if len(review) > 2:
            try:
                loss = train(review[:-1].cuda(), review[1:].cuda())       
            except:
                print(review, len(review))
                raise
                
            loss_avg += loss

            if i % print_every == 0:
                print('[(%d %d%%) %.4f]' % (epoch, epoch / n_epochs * 100, loss))
                print(generate_chunk(predict_len = 50), '\n')

[(1 0%) 4.8674]
O produto.H;@n>V3jx99[H^T9^iiJ iHv x>zaV3h|} 

[(1 0%) 1.9085]
O produto ente em produto pra mazo paras e mas perado as en 

[(1 0%) 1.8718]
O produto o o e muito a comprado produto comprela recebinto 

[(1 0%) 2.0888]
O produto compro e perecebi entregar entregue e produto no  

[(1 0%) 1.3835]
O produto nao estar a prazo comendo e compreio ou produto e 

[(1 0%) 1.2181]
O produto produto e teu ser em para desto essa des estimo e 

[(1 0%) 0.7465]
O produto de entes do chegou chegou antrega de produto emba 

[(1 0%) 1.3504]
O produto entrega e ma perfeito parabendo ponta do par cali 

[(1 0%) 1.0470]
O produto com em produto sempre e produto produto o produto 

[(1 0%) 2.7963]
O produto com do prazo e a que produto entregue um pora o f 

[(1 0%) 0.8446]
O produto chegara estava e de prelo e e esta e tinha compre 

[(1 0%) 2.5005]
O produto e produto dentregar o prazo e comprei produto e p 

[(1 0%) 1.4100]
O produto e baracao tem em pora comendo correto 

[(1 0%) 0.2711]
O produto e em perfeita. Muito obrigado. Recomendo. E produ 

[(1 0%) 0.8135]
O produto chegou antes do prazo de entrega e em perfeito e  

[(1 0%) 1.0087]
O produto e bem embalado e pelo que eu faltando os partes,  

[(1 0%) 0.9999]
O produto foi entregue dentro do prazo estipulado e de otim 

[(1 0%) 0.3749]
O produto chegou no prazo, e entrega rapida, produto no meu 

[(1 0%) 0.7603]
O produto com defeito. Esta foi entregue dentro do prazo. P 

[(1 0%) 1.3663]
O produto e com a entrega foi antes do prazo. Estou muito s 

[(1 0%) 0.9611]
O produto de qualidade e entrega antes do prazo estimado. B 

[(1 0%) 0.5881]
O produto de maneira e nao foi entregue dentro do prazo e p 

[(1 0%) 0.6664]
O produto nao foi entregue antes do prazo e estou aguardand 

[(1 0%) 0.4090]
O produto chegou em contato com uma compra e recebi apenas  

[(1 0%) 1.1702]
O produto e entrega dentro do prazo!!!!!!!!!!!!!!!!!!!!!!!! 

[(1 0%) 1.6903]
O produto chegou antes do prazo estipulado pelo 

In [ ]:
torch.save(lstm.state_dict(), "lstm.pth")